In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv

--2023-10-01 10:30:51--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1.4M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.41M  1.49MB/s    in 0.9s    

2023-10-01 10:30:53 (1.49 MB/s) - ‘data.csv’ saved [1475504/1475504]



In [274]:
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score

In [286]:
data=pd.read_csv('data.csv')

# Data preparation


In [287]:
variables=['Make',
'Model',
'Year',
'Engine HP',
'Engine Cylinders',
'Transmission Type',
'Vehicle Style',
'highway MPG',
'city mpg','MSRP']
data=data[variables]
columns=[i.lower().replace(" ","_") for i in data.columns] 
data.columns=columns
data=data.fillna(0)
data=data.rename(columns={'msrp':'price'})
# categorical=['make','model','transmission_type','vehicle_style']
# numerical=['year','engine_hp','engine_cylinders','highway_mpg','city_mpg','price']


,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500
...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,46120
11910,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,56670
11911,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50620
11912,Acura,ZDX,2013,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50920


# Question 1
What is the most frequent observation (mode) for the column transmission_type? <br>

AUTOMATIC <br>
MANUAL <br>
AUTOMATED_MANUAL <br>
DIRECT_DRIVE <br>

In [288]:
data['transmission_type'].value_counts()

transmission_type
AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: count, dtype: int64

# Question 2
Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset. <br>

What are the two features that have the biggest correlation in this dataset?<br>

engine_hp and year<br>
engine_hp and engine_cylinders<br>
highway_mpg and engine_cylinders<br>
highway_mpg and city_mpg<br>
Make price binary<br>
Now we need to turn the price variable from numeric into a binary format.<br>
Let's create a variable above_average which is 1 if the price is above its mean value and 0 otherwise.<br>
Split the data<br>
Split your data in train/val/test sets with 60%/20%/20% distribution.<br>
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.<br>
Make sure that the target value (above_average) is not in your dataframe.

In [289]:
numeric=['engine_hp','year','engine_cylinders','highway_mpg','city_mpg']
data[numeric].corr()

,engine_hp,year,engine_cylinders,highway_mpg,city_mpg
engine_hp,1.000000,0.338714,0.774851,-0.415707,-0.424918
year,0.338714,1.000000,-0.040708,0.258240,0.198171
engine_cylinders,0.774851,-0.040708,1.000000,-0.614541,-0.587306
highway_mpg,-0.415707,0.258240,-0.614541,1.000000,0.886829
city_mpg,-0.424918,0.198171,-0.587306,0.886829,1.000000


In [290]:
# get the price binary
mean=data['price'].mean()
data['above_average']=(data['price']>mean).astype('int')

In [291]:
# split the data
train,test=train_test_split(data,test_size=.4,random_state=42)
test,val=train_test_split(test,test_size=.5,random_state=42)
# reset index
train=train.reset_index(drop=True)
test=test.reset_index(drop=True)
val=val.reset_index(drop=True)
# y_variable
y_train=train['above_average'].values
y_test=test['above_average'].values
y_val=val['above_average'].values
# delete that variable from data we have
del train['above_average']
del test['above_average']
del val['above_average']

# Question 3
Calculate the mutual information score between above_average and other categorical variables in our dataset. Use the training set only.<br>
Round the scores to 2 decimals using round(score, 2).<br>
Which of these variables has the lowest mutual information score?<br>

make<br>
model<br>
transmission_type<br>
vehicle_style

In [292]:
# because apply require one argument we create a function
def mutual(serie):
    return mutual_info_score(serie,y_train)
round(train[categorical].apply(mutual).sort_values(ascending=False),2)

model                0.46
make                 0.24
vehicle_style        0.08
transmission_type    0.02
dtype: float64

# Question 4
Now let's train a logistic regression.<br>
Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.<br>
Fit the model on the training dataset. <br>
To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:<br>
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
Calculate the accuracy on the validation dataset and round it to 2 decimal digits.<br>
What accuracy did you get?<br>

0.60<br>
0.72<br>
0.84<br>
0.95

In [303]:
# on train
dicts_train=train.to_dict(orient='records')
dv=DictVectorizer(sparse=False)
dv.fit(dicts_train)
x_train=dv.transform(dicts_train)
# on test

dicts_test=test.to_dict(orient='records')
x_test=dv.transform(dicts_test)

#on val

dicts_val=val.to_dict(orient='records')
x_val=dv.transform(dicts_val)


In [323]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(x_train,y_train)
predict=model.predict(x_test)
accuracy_score(predict,y_test)

0.9991607217792698

In [310]:
x_test.shape,x_train.shape

((2383, 960), (7148, 960))